<a href="https://colab.research.google.com/github/gabrielvieiraf/ProjetosPython/blob/master/AltaPerformance/Introdu%C3%A7%C3%A3o/Introdu%C3%A7%C3%A3o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Este notebook é dedicado para aqueles que visam desenvolver suas habilidades em Python, para quem deseja consultar alguns conceitos perdidos na memória e para programadores que vêm de outra linguagem, principalmente de mais baixo nível, e se deparam com a pergunta: **Existe performance no Python?**.

Este notebook será baseado no livro *High Performance Python, 2nd Edition* e tem, por sua vez, comentários e alterações feitas por mim, **Gabriel Vieira Flores**, afim de deixar o conteúdo com um pouco mais de informação e didático para os falantes da língua portuguesa.

> # Introdução ao Python Performático

Para trabalhar um código de alta performance em Python, devemos ter em mente as três partes de um sistema computacional: 
**Unidade de Computação, Unidade de Memória e a junção destas.**


> ## Unidade de Computação


  A unidade de computação de um computador é responsável pela capacidade de manipulação de *bits*. 

> ### CPU & GPU


  As **CPU'S** são mais usadas em unidades de computação, entretanto as **GPU's** estão ganhando popularidade como unidades auxiliares para a computação, acelarando a **computação gráfica**, mas estão sendo úteis também para realizar **inúmeros cálculos feitos de maneira paralela**. 

  As principais propriedades em uma unidade de computação são o **número de operações que pode fazer em um ciclo** e o **número de ciclos que pode fazer em um segundo.**

  Além disso, embora o aumento da velocidade do *clock* acelere todos os programas em execução nessa unidade computacional (porque eles são capazes de fazer mais cálculos por segundo), ter um IPC mais alto também pode afetar drasticamente a computação, alterando o nível de vetorização que é possível.

 A vetorização ocorre quando uma **CPU é fornecida com várias partes de dados de uma vez** e pode operar em todos eles ao mesmo tempo.
A instrução da CPU é conhecida como instrução única, dados múltiplos (SIMD).


> ### Velocidade de *clock*


  As velocidades de *clock* e IPC têm estado estagnadas por causa das limitações físicas de fazer transistores cada vez menores. Como resultado, os fabricantes de chips têm confiado em outros métodos para ganhar mais velocidade, incluindo *multithreading* simultâneo (onde vários *threads* podem ser executados ao mesmo tempo).

*Hyperthreading* apresenta uma segunda CPU virtual para o sistema operacional *host*, e a lógica de *hardware* inteligente tenta intercalar dois *threads* de estruturas nas unidades de execução em uma única CPU. Quando bem-sucedido,
ganhos de até 30% sobre um único segmento podem ser alcançados.

Funciona bem quando as unidades de trabalho em ambos os *threads* usam tipos diferentes de unidades de execução - por exemplo, um realiza operações de ponto flutuante *(float)* e o outro executa operações inteiras *(int)*.



> ### Arquitetura *Multicore*


  Essas arquiteturas incluem vários CPUs dentro da mesma unidade, o que aumenta a capacidade total para tornar cada unidade individual mais rápida. É por isso que atualmente é difícil encontrar qualquer máquina com menos de dois núcleos - neste caso, o computador tem duas unidades físicas de computação que estão conectadas uns aos outros. Enquanto isso aumenta o número total de operações que podem ser feitas por segundo, contudo, pode tornar a escrita de código mais difícil!

> ### Tempo de execução e a Lei de Amdahl


Apenas adicionar mais núcleos a uma CPU nem sempre acelera o desempenho de um programa. Isso se deve a algo conhecido como **Lei de Amdahl:**

  **" Se um programa projetado para ser executado em núcleos múltiplos tem algumas sub-rotinas que devem ser executadas em um núcleo, este será a limitação para a aceleração máxima que pode ser alcançada pela alocação de mais núcleos."**

**Por exemplo:** Se tivéssemos uma pesquisa que quiséssemos que cem pessoas preenchessem, e essa pesquisa levou 1 minuto para ser concluída, poderíamos concluir esta tarefa em 100 minutos se tivéssemos uma pessoa fazendo as perguntas (ou seja, essa pessoa vai para o participante 1, faz as perguntas, espera pelas respostas e então passa para o participante 2).

Este método de ter uma pessoa perguntando e esperar por respostas é semelhante a um **processo serial**. Em processos em série, temos operações sendo satisfeitas **uma de cada vez**, cada uma espera para que a operação anterior seja concluída.
No entanto, poderíamos realizar a pesquisa em **paralelo** se tivéssemos duas pessoas fazendo as perguntas, o que nos deixaria terminar o processo em apenas 50 minutos útes. Isso pode ser feito porque cada pessoa que faz as perguntas
não precisa saber nada sobre a outra pessoa que está fazendo perguntas.
Como resultado, a tarefa pode ser facilmente dividida sem ter qualquer dependência entre os questionadores.

Adicionar mais pessoas fazendo as perguntas **nos dará mais acelerações**,
até que tenhamos cem pessoas fazendo perguntas. Neste ponto, o processo
levaria 1 minuto e seria limitado simplesmente pelo tempo que leva um
participante para responder a perguntas. Adicionar mais pessoas fazendo perguntas não resultará em qualquer aumento de velocidade, porque essas pessoas extras não terão tarefas a serem executadas **- todos os participantes já estão respondendo a perguntas!**

Da mesma forma, com CPUs, podemos adicionar mais núcleos que podem realizar vários blocos de computação conforme necessário até chegarmos a um ponto em que o gargalo é o **tempo que leva para um núcleo específico para terminar sua tarefa**.

Logo, podemos dizer que o gargalo em qualquer **cálculo paralelo** são sempre as tarefas seriais menores que estão sendo distribuídas.


> ### Python & GIL


Um grande obstáculo com a utilização de vários núcleos em Python é o uso de um bloqueio de intérprete global (GIL).

O *Global Interpreter Lock* é uma flag que existe no interpretador do Python, e faz com que apenas uma sequência de *bytecode* na VM de Python seja executada por vez.

A razão inicial de sua criação é que o gerenciamento interno de memória do interpretador Python não é *thread-safe*.

**OBS:** *Thread safety* (segurança de fio de execução) é um conceito aplicável no contexto de programas *multi-thread*. Um pedaço de código é dito *thread-safe* se ele apenas manipula estruturas de dados compartilhadas de uma forma que garanta uma execução segura através de várias *threads* ao mesmo tempo.

**Isso é:** O código em C que reserva memória para criação de objetos, mesmo os simples como inteiros ou tuplas, usa o GIL como Mutex para garantir que não vai ser interrompido pelo sistema operacional, numa mudança de *thread*, e deixando as estruturas de dados internas que o Python usa para memória num estado inconsistente.

Além disso, O GIL é o responsável pelas estruturas de dados do Python como listas, dicionários e, mais importante, as estruturas internas como *frames* de execução, os dicionários que armazenam variáveis, etc. Funcionem de forma transparente em código que use várias *threads*.

Basicamente, o código central da VM do Python, a estrutura de "dispatch" que é um grande "switch case" para o *bytecode* só "anda" se tiver controle do GIL. 

Usando o exemplo anterior de uma pesquisa, isso significaria que mesmo se tivéssemos 100 questionadores, apenas uma pessoa pode fazer uma pergunta e ouvir uma resposta de cada vez. Este efetivamente remove qualquer tipo de benefício de ter várias perguntas pedintes!

Embora isso possa parecer um grande obstáculo, este problema pode ser evitado usando outra biblioteca padrão de ferramentas, como numpy, numexpr, Cython ou modelos distribuídos.

> ## Unidade de Memória

  

As unidades de memória em computadores são usadas para armazenar *bits*. 
Podem ser *bits* que representam variáveis em seu programa ou *bits* que
representam os pixels de uma imagem.

Assim, a abstração de uma unidade de memória se aplica aos registros em sua
placa-mãe, bem como sua RAM e disco rígido.

> ### Diferença entre tipos de unidades de memória

A única diferença principal entre todos esses tipos de unidades de memória é a
velocidade com que eles podem ler/gravar dados. Para tornar as coisas mais complicadas, a velocidade de leitura/gravação depende muito da maneira com que os dados estão sendo lidos.

Por exemplo, a maioria das unidades de memória tem um desempenho muito melhor quando lêem um grande pedaço de dados em oposição a muitos pequenos pedaços (isso é conhecido como leitura sequencial versus dados aleatórios). Se os dados nessas unidades de memória forem pensados como páginas de um grande livro, isso significa que a maioria das unidades de memória têm melhores velocidades de leitura / gravação ao passar pelo livro página por página em vez de mudar constantemente de uma página aleatória para outra.

Enquanto este fato é geralmente verdadeiro em todas as unidades de memória, a quantidade é drasticamente diferente.
Além das velocidades de leitura / gravação, as unidades de memória também têm latência, que pode ser caracterizado como o tempo que o dispositivo leva para encontrar os dados que está sendo usado. Para um disco rígido girando, essa latência pode ser alta acima de fazer com que o disco gire fisicamente até atingir a velocidade e a cabeça de leitura deve mover para a posição certa. Por outro lado, para RAM, essa latência pode ser bem pequena porque tudo é de estado sólido. 


Aqui está uma breve descrição das várias unidades de memória que são comumente encontradas dentro de um padrão estação de trabalho, em ordem de velocidade de leitura / gravação:

> ### **Disco rígido girando**
Armazenamento de longo prazo que persiste mesmo quando o computador é desligado.
Geralmente tem velocidades lentas de leitura / gravação porque o disco deve ser físico
cally girou e se moveu. Desempenho degradado com acesso aleatório padrão
terns, mas uma capacidade muito grande (intervalo de 10 terabytes).

> ### **Disco rígido de estado sólido**
Semelhante a um disco rígido giratório, com velocidades de leitura / gravação mais rápidas, mas capacidade menor (intervalo de 1 terabyte).

> ### **RAM**
Usado para armazenar o código do aplicativo e dados (como quaisquer variáveis ​​sendousava). Tem características de leitura / gravação rápidas e funciona bem com padrões de acesso dom, mas geralmente tem capacidade limitada (64 gigabytes
alcance).

> ### **Cache L1 / L2**
Velocidades de leitura / gravação extremamente rápidas. Os dados que vão para a CPU devem ir por aqui. Capacidade muito pequena (faixa de megabytes).

**Uma tendência claramente visível é que as velocidades de leitura / gravação e a capacidade são inversamente proporcionais - conforme tentamos aumentar a velocidade, a capacidade é reduzida.**

Muitos sistemas implementam uma abordagem em camadas para a memória: os dados começam em seu estado completo no disco rígido, parte dele se move para a RAM e, em seguida, muito subconjunto menor se move para o cache L1 / L2. Este método de classificação permite programas para manter a memória em lugares diferentes, dependendo da velocidade de acesso requisitos.

**Ao tentar otimizar os padrões de memória de um programa, estamos simplesmente otimizando quais dados são colocados, onde, como são dispostos (em para aumentar o número de leituras sequenciais) e quantas vezes é movido entre os vários locais.**
